# Functions for computing the elasticity of a sumset semigroup

Loading the CommutativeMonoids library. You can download with the command 
```properties
$ git clone https://github.com/D-marina/CommutativeMonoids.git
```
The only library use is `integerSmithNormalFormAndApplications` which is in the folder `Class`

In [1]:
import sys
#location of the folder CommutativeMonoids
# we load the following libraies
pathToCommutativeMonoids='..'
sys.path.insert(0,pathToCommutativeMonoids+'/ClassCSemigroup')
sys.path.insert(0,pathToCommutativeMonoids+'/Class')
sys.path.insert(0,pathToCommutativeMonoids+'/ClassAffine')
sys.path.insert(0,pathToCommutativeMonoids+'/CClass')

from integerSmithNormalFormAndApplications import *

We load now all the other libraries we also need and define in sympy new symbols

In [2]:
import functools,math
import sympy
import numpy as np
from PyNormaliz import *
from numpy import array
from sympy import symbols,groebner
from fractions import Fraction

x1,x2,x3,x4,x5=symbols("x1 x2 x3 x4 x5")
lX=[x1,x2,x3,x4,x5]
y1,y2,y3,y4,y5=symbols("y1 y2 y3 y4 y5")
lY=[y1,y2,y3,y4,y5]
z1,z2,z3,z4,z5=symbols("z1 z2 z3 z4 z5")
lZ=[z1,z2,z3,z4,z5]
w1,w2,w3,w4,w5=symbols("w1 w2 w3 w4 w5")
lW=[w1,w2,w3,w4,w5]

x,y,z,t,u,v,w=symbols("x y z t u v w")

The operator $+$ and $\otimes$ are defined below

In [3]:
class Infix:
    def __init__(self, function):
        self.function = function
    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __or__(self, other):
        return self.function(other)
    def __rlshift__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __rshift__(self, other):
        return self.function(other)
    def __call__(self, value1, value2):
        return self.function(value1, value2)

@Infix
def add(A,B):
    '''Computes the addition of two sets A and B
    >>> [7,9] |add| [0,2,3]
    '''
    C=list(set([a+b for a in A for b in B]))
    C.sort()
    return C

@Infix
def mult(a,A):
    '''Sum with itself a-times the set A
    >>> 3 |mult| [3,4]
    '''
    if a==0:
        return [0]
    elif a==1:
        A=list(set(A))
        A.sort()
        return A
    else:
        return A |add| mult(a-1,A)

We define now the functions for computing the presentation of sumset semigroup of the form of Algorithm 1 of [On the ideals of some sumset semigroups](https://arxiv.org/abs/2102.04100). 

The main function if `computationIS` and it also checks if the given list of generators fulfill the requeriments of Algorithm 1 of [On the ideals of some sumset semigroups](https://arxiv.org/abs/2102.04100).

An example of how to use this funcion is 
```properties
>>> computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
```

In [4]:
def computeSets(lsgS):
    '''Computes some auxiliary sets used in functino computationIS
    '''
    lB=[]
    lA=[]
    la=[]
    for S in lsgS:
        if len(S)==1:
            if S not in lB:
                lB.append(S)
        else:
            a0=S[0]
            if a0!=0 and [a0] not in la:
                la.append([a0])
            aux=(a0,[a-a0 for a in S])
            if aux not in lA:
                lA.append(aux)
    lkA=[S[1][0:2] for S in lA]
    lkA=[S for i,S in enumerate(lkA) if S not in lkA[:i]]
    return lB,la,lA,lkA

def computeNM(S,lA):
    '''Computes a decomposition of S in term of the elements of lA
    >>> computeNM([0,2,5,7],[[0,5],[0,2]])
    '''
    for i in range(S[-1]//lA[0][-1]+1):
        for j in range(S[-1]//lA[1][-1]+1):
            if S==(i|mult|lA[0]) |add| (j|mult|lA[1]):
                return (i,j)

def computationIS(*sgS):
    '''Computes the ideal of the sumset semigroup generated by sgS
    >>> computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
    '''
    lB,la,lA,lkA=computeSets(list(sgS))
    
    aux=[T[1][:2] for T in lA]
    aux1=[S for i,S in enumerate(aux) if S not in aux[:i]]
    if len(aux1)!=2:
        print("No puedo calcular presentación")
        return
    Sprime=lB+la+aux1
    S1=lB+la
    
    lnm=[(S[0],S[1],computeNM(S[1],lkA)) for S in lA]
    
    sn=[S[0] for S in lB+la]
    a,b=lkA[0][1],lkA[1][1]
    k=math.gcd(a,b)
    a,b=a//k,b//k
    relations=[u**S[0]-lX[j] for j,S in enumerate(lB)]
    relations=relations+[u**S[0]-lW[j] for j,S in enumerate(la)]
    relations=relations+[x**(b-1)*y**(a-1)*(x**b-y**a)]
    for j,TT in enumerate(lnm):
        ai,Anm,coord=TT
        if ai!=0:
            pos=[aa[0] for aa in la].index(ai)
            relations=relations+[lZ[j]-lW[pos]*x**coord[0]*y**coord[1]]
        else:
            relations=relations+[lZ[j]-x**coord[0]*y**coord[1]]
    
    gb=groebner(relations,u,x,y,*lW,*lX,*lZ)
    
    gb1=[p for p in gb \
         if set(p.args[0].as_powers_dict().keys()).intersection([u,x,y]+lW)==set()\
         and \
         set(p.args[1].as_powers_dict().keys()).intersection([u,x,y]+lW)==set()]
    
    return gb1


In [5]:
gb=computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
gb

⎡  4     3    3   3   2     3    3           2   2    3           2          2
⎣x₁  - x₂ , x₁ ⋅x₂ ⋅z₃  - z₂ , x₁ ⋅z₁⋅z₂ - x₂ ⋅z₂ , x₁ ⋅z₁⋅z₃ - x₂ ⋅z₂⋅z₃, x₁ 

   3   3     3       2   2           3    2   2           2          4   2    
⋅x₂ ⋅z₃  - z₁ ⋅z₃, x₁ ⋅z₁ ⋅z₂ - x₂⋅z₂ , x₁ ⋅z₁ ⋅z₃ - x₂⋅z₂ ⋅z₃, x₁⋅x₂ ⋅z₃  - z

 2          3        4       3        3          2                            
₁ ⋅z₂, x₁⋅z₁ ⋅z₂ - z₂ , x₁⋅z₁ ⋅z₃ - z₂ ⋅z₃, x₁⋅z₂  - x₂⋅z₁⋅z₂, x₁⋅z₂⋅z₃ - x₂⋅z

        5   2        2    4   3   2     5       4   2   3     5       3   7   
₁⋅z₃, x₂ ⋅z₃  - z₁⋅z₂ , x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₂, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₃, x₂ ⋅z₂ ⋅z₃

2     9       3   6   3     9       2   11   2     13       2   10   3     13 
  - z₁ ⋅z₂, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₃, x₂ ⋅z₂  ⋅z₃  - z₁  ⋅z₂, x₂ ⋅z₂  ⋅z₃  - z₁  ⋅

         4        5       4        4          15   2     17          14   3   
z₃, x₂⋅z₁ ⋅z₂ - z₂ , x₂⋅z₁ ⋅z₃ - z₂ ⋅z₃, x₂⋅z₂  ⋅z₃  - z₁  ⋅z₂, x₂⋅z₂  ⋅z₃  - 

  17       21        19   2    21        18   

Now, we define three auxiliary funcions `variablesGB`, `fraccionDeGenerador` and `binomio2Zn` used in the function `elasticity`. The last function computes the elasticity of a monoid from one of its defining ideals.

In [6]:
def variablesGB(gb):
    '''
    >>> variablesGB(gb)
    '''
    lV=[]
    for p in gb:
        lV=lV+list(p.args[0].as_powers_dict().keys())
        lV=lV+list(p.args[1].as_powers_dict().keys())
    return set(lV).difference([-1])

def fraccionDeGenerador(gen):
    dim=len(gen)//2
    #print(gen,dim)
    return Fraction(sum(gen[:dim]),sum(gen[dim:]))

def binomio2Zn(binomio,lV):
    #binomio=G4[0]
    s1,s2=binomio.args[0].as_powers_dict(),binomio.args[1].as_powers_dict()
    c=[]
    c1=[]
    c2=[]
    #print(s1)
    for v in lV:
        c1.append(s1[v])
    for v in lV:
        c2.append(s2[v])
    return [c1,c2]

We have the function elasticity

In [7]:
def elasticity(gb,showTuples=False):
    '''Computes the elasticity of a monoid from one of its presetations.
    It returns the value of the elasticity and the elements of gb that reach the elasticity
    >>> rho,lTElasticity=elasticity(gb)
    '''
    tuplasGB=[binomio2Zn(b,variablesGB(gb)) for b in gb]
    if showTuples:
        print(tuplasGB)
    genM=[]
    for c1,c2 in tuplasGB:
        genM.append(array(c1)-array(c2))

    eqGenM=generatorsToEquations(genM)
    numEqs=eqGenM[0].shape[0]
    filas=[list(eqGenM[0][i,:]) for i in range(numEqs)]

    ecuacionesAmenosA=[filas[i]+[-a for a in filas[i]] for i in range(numEqs)]
    monoide=Cone(equations=ecuacionesAmenosA)
    sistemaGenMonoide=monoide.HilbertBasis()
    
    dim=len(sistemaGenMonoide[0])//2
    elasticidad=max([fraccionDeGenerador(gen) for gen in sistemaGenMonoide])
    
    elasticidad

    Rs=[gen for gen in sistemaGenMonoide \
     if fraccionDeGenerador(gen)==elasticidad]
    return(elasticidad,Rs)
    

One example of how to use this function is the following.

In [8]:
gb=computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
rho,lTElasticity=elasticity(gb)
print(rho)

3


From the work [Atomic Commutative Monoids and Their Elasticity
](https://doi-org.bibezproxy.uca.es/10.1007/s00233-002-0022-4), we can check from a presetation of a monoid if it has acceptable elasticity or not.

In [9]:
def supportIncluded(vect1,vect2):
    '''
    >>> supportIncluded([0,0,3],[1,0,1])
    '''
    for i,j in zip(vect1,vect2):
        if i!=0 and j==0:
            return False
    return True


def hasAcceptableElasticity(gb,Rs,showTuple=False):
    '''From one of its presentations, computes if the elasticity of a monoid is acceptable
    >>> hasAcceptableElasticity(gb,lTElasticity)
    '''
    tuplasGB=[binomio2Zn(b,variablesGB(gb)) for b in gb]
    ts=[vv[0]+vv[1] for vv in tuplasGB]
    for vv in ts:
        for ww in Rs:
            if supportIncluded(vv,ww):
                if showTuple:
                    print("Support of",vv,"included in the support of",ww)
                return True
    return False

## Example of sumset monoid without acceptable elasticity

The monoid considered is $\langle \{3\},\{4\},\{6,12\},\{7,10,13\},\{0,3,6,9\} \rangle$

In [10]:
gb=computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
rho,lTElasticity=elasticity(gb)
print("Elasticity is",rho)
hasAcceptableElasticity(gb,lTElasticity)

Elasticity is 3


False

## Example of sumset monoid with acceptable elasticity

In this example the monoid is $\langle \{3\},\{4\},\{6,12\},\{7,10,13\} \rangle$

In [11]:
gb=computationIS([3],[4],[6,12],[7,10,13])
gb

⎡  4     3    3           2   2    2   2           3       3        4       2 
⎣x₁  - x₂ , x₁ ⋅z₁⋅z₂ - x₂ ⋅z₂ , x₁ ⋅z₁ ⋅z₂ - x₂⋅z₂ , x₁⋅z₁ ⋅z₂ - z₂ , x₁⋅z₂  

                 4        5⎤
- x₂⋅z₁⋅z₂, x₂⋅z₁ ⋅z₂ - z₂ ⎦

In [12]:
rho,lTElasticity=elasticity(gb,True)
rho,lTElasticity

[[[0, 0, 4, 0], [0, 0, 0, 3]], [[0, 2, 0, 2], [1, 1, 3, 0]], [[0, 3, 0, 1], [2, 1, 2, 0]], [[0, 4, 0, 0], [3, 1, 1, 0]], [[0, 2, 1, 0], [1, 1, 0, 1]], [[0, 5, 0, 0], [4, 1, 0, 1]]]


(Fraction(4, 3),
 [[0, 0, 4, 0, 0, 0, 0, 3],
  [1, 0, 3, 0, 0, 1, 0, 2],
  [2, 0, 2, 0, 0, 2, 0, 1],
  [3, 0, 1, 0, 0, 3, 0, 0]])

In [13]:
hasAcceptableElasticity(gb,lTElasticity,True)

Support of [0, 0, 4, 0, 0, 0, 0, 3] included in the support of [0, 0, 4, 0, 0, 0, 0, 3]


True